
# сравнение функций потерь от сжатого и сырого контекстов.

### TODO
* Отладка. Проблема Сжатый контекст: N/A
* Пересмотреть функцию потерь, чтобы она отражала задачу сжатия более точно (например, использовать потери восстановления или схожести между оригиналом и сжатым контекстом).
* Усложнить архитектуру модели ContextOptimizer, добавив больше слоев или более продвинутые блоки, такие как GRU или LSTM.

#### Основная цель
Обучить отдельную сеть \( O \) (модуль оптимизации контекста), которая динамически изменяет контекст таким образом, чтобы предсказания исходной языковой модели \( M \) оставались точными, несмотря на уменьшение объема информации в контексте.

### Структура и Процесс Обучения

1. **Основная Языковая Модель \( M \):**
   - \( M \) — существующая языковая модель, такая как GPT-2, которая принимает на вход полный контекст и текущий фрагмент и генерирует предсказания.
   - Цель \( M \) — максимальная точность предсказаний на основе полного контекста.

2. **Модуль Оптимизации Контекста \( O \):**
   - Модуль \( O \) используется для сжатия предыстории \( C \) с учетом текущего фрагмента \( S \), чтобы создать оптимизированный контекст \( C' \).
   - Модуль \( O \) может быть реализован через нейронные сети, такие как трансформеры, рекуррентные сети или другие архитектуры, способные выделять ключевые элементы контекста.

3. **Целевая Функция и Метрики:**
   - Основной задачей является минимизация расхождения между предсказаниями модели \( M \) на основе \( C' \) и \( C \). Целевая функция может быть определена как:
     \[
     L = \text{Loss}(M(C', S), M(C, S))
     \]
   - Для измерения качества сжатия можно использовать метрики, такие как среднеквадратичная ошибка (MSE) между эмбеддингами предсказаний, кросс-энтропия, и т.д.
   - Важно также отслеживать сходимость потерь между \( C' \) и \( C \), чтобы убедиться, что \( C' \) минимально влияет на точность предсказаний.

4. **Процесс Обучения:**
   - **Шаг 1:** Из датасета создаются пары \( (C, S, T) \), где \( C \) — предыстория, \( S \) — текущий фрагмент, и \( T \) — целевой текст для предсказания.
   - **Шаг 2:** Модуль \( O \) сжимает контекст \( C \), производя \( C' = O(C, S) \).
   - **Шаг 3:** Модель \( M \) предсказывает на основе \( (C', S) \) и \( (C, S) \). Сравниваются предсказания для оценки потерь.
   - **Шаг 4:** Потери используются для обновления параметров модуля \( O \), минимизируя влияние сжатия на точность.

5. **Анализ и Диагностика:**
   - Постепенно выводим промежуточные результаты для диагностики, включая анализ предсказаний по сжатому и полному контексту.
   - Выполняем оценку примеров, где предсказания на основе сжатого контекста значительно отличаются от предсказаний на основе полного контекста.
   - Используем фильтрацию данных для обучения, например, исключаем случаи, где текущий фрагмент уже достаточен для точного предсказания.

### Рекомендации для Улучшения

- **Итеративная Диагностика:** Важно регулярно проверять, насколько сжатый контекст \( C' \) сохраняет критическую информацию. Диагностика на нескольких примерах может выявить случаи, где \( O \) недооценивает важные части контекста.
- **Контроль за Переобучением:** Следите за тем, чтобы модель \( O \) не начинала генерировать «фантазии» или добавлять избыточные детали в попытках компенсировать потерю информации.
- **Улучшение Сходимости:** Добавьте регуляризацию или дополнительные ограничения на модель \( O \), чтобы предотвратить отклонения в сжатии, которые могут ухудшить точность предсказаний.

### Ожидаемый Результат

- Модуль \( O \) научится выделять только те части контекста, которые критически важны для точных предсказаний, избегая ненужного увеличения объема информации.
- Модель \( M \), используя \( C' \), будет предсказывать с минимальной потерей точности, что укажет на успешное сжатие и оптимизацию контекста. 


### 1. **Подготовка Данных**
   - **Предварительная обработка данных:** Датасет разбивается на последовательности: предыстория (context), текущий фрагмент (current_fragment), и целевой текст (target).
   - **Фильтрация данных:** Удаляются пустые и нерелевантные записи, например, те, где текущий фрагмент не вносит значимого вклада в предсказание.
   - **Предварительная оценка потерь:** Вычисляются потери для различных комбинаций контекста, что позволяет заранее оценить вклад контекста в точность предсказания.

### 2. **Оптимизация Контекста**
   - **Целевая функция:** Используется MSELoss для измерения расхождения между предсказаниями модели и целевыми значениями.
   - **Сходимость потерь:** Основная цель — сходимость потерь сжатого контекста (compressed_loss) к потерям сырого контекста (raw_loss).
   - **Контроль за «додумыванием»:** Введены штрафы за значительное отклонение между потерями сжатого и сырого контекста, чтобы предотвратить генерацию ложных зависимостей.

### 3. **Адаптивная Регуляция Потерь**
   - **Штрафы за расхождения:** Если потери сжатого контекста значительно отличаются от потерь сырого, в потери добавляется штраф, который стимулирует модель уменьшать эти различия.
   - **Адаптивное обучение:** Обучение адаптируется на основе расхождений, улучшая способность модели сжимать контекст, сохраняя при этом точность.

### 4. **Использование Промежуточных Датасетов**
   - **Сохранение промежуточных результатов:** Включение предсказаний модели в промежуточный датасет, чтобы ускорить и упростить анализ, отладку и последующую оптимизацию.
   - **Фильтрация на этапе подготовки:** Отбираются записи, в которых контекст оказывает значительное влияние на предсказания, отбрасывая те, где контекст несущественен.

### 5. **Визуализация и Отладка**
   - **Графики потерь:** Построение графиков с использованием Plotly для отслеживания динамики потерь сжатого и сырого контекстов.
   - **Примеры для анализа:** Регулярный вывод примеров с различными типами предсказаний (сжатый контекст, сырой контекст, текущий фрагмент) для визуального анализа и проверки корректности работы модели.

### 6. **Управление Временем и Эффективностью**
   - **Оценка времени:** Оценка времени выполнения подготовки данных и каждой эпохи обучения на основе первых пяти циклов, что позволяет планировать ресурсозатраты.
   - **Оптимизация кода:** Фокусировка на ключевых этапах, таких как загрузка моделей, настройка параметров и запуск оптимизации с минимальными затратами времени.

### Основные цели:
- Обеспечение качественного сжатия контекста без потери точности.
- Сведение к минимуму расхождения потерь между сжатым и сырым контекстами.
- Избежание нежелательных интерпретаций и «додумывания» контекста моделью.

## Модульная структура

In [25]:
# Вспомогательные классы и методы

import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch.nn as nn
import plotly.graph_objects as go
from IPython.display import display
import os
import json


# Подготовка данных
def prepare_data(dataset, max_length=300):
    """
    Функция подготавливает данные из датасета, включая предысторию, текущий фрагмент и целевой фрагмент текста,
    используя смещающееся окно по элементам датасета.

    :param dataset: Датасет с текстами.
    :param max_length: Максимальная длина текущего фрагмента.
    :return: Список кортежей (предыстория, текущий фрагмент, целевой фрагмент).
    """
    prepared_data = []
    buffer = []
    counter = 0
    for example in dataset:
        text = example["text"].strip()

        # Пропускаем пустые записи
        # TODO Убрать отладочную выборку первых 1000 записей
        if len(text) == 0 or counter > 1000:
            continue
        counter += 1
        buffer.append(text)

        # Если буфер содержит три фрагмента, создаем (context, current_fragment, target)
        if len(buffer) == 3:
            context, current_fragment, target = buffer

            # Обрезаем фрагменты по max_length, если необходимо
            context = context[:max_length].strip()
            current_fragment = current_fragment[:max_length].strip()
            target = target[:max_length].strip()

            # Пропускаем записи, если какой-либо из фрагментов пустой
            if all([context, current_fragment, target]):
                prepared_data.append((context, current_fragment, target))

            # Сдвигаем окно
            buffer.pop(0)
    return prepared_data


# Функции для сохранения и загрузки модели
def save_model(model, optimizer, path="context_optimizer.pth"):
    torch.save(
        {
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        path,
    )
    print(f"Model saved to {path}")


def load_model(model, optimizer, path="context_optimizer.pth"):
    if os.path.exists(path):
        checkpoint = torch.load(path)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        print(f"Model loaded from {path}")
    else:
        print(f"Model file not found at {path}")


# Визуализация с использованием Plotly FigureWidget
def create_loss_plot():
    fig = go.FigureWidget()
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Raw Context Loss"))
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Compressed Context Loss"))
    fig.add_trace(go.Scatter(x=[], y=[], mode="lines+markers", name="Fragment Only Loss"))
    fig.update_layout(title="Графики потерь во время обучения", xaxis_title="Эпоха", yaxis_title="Среднее значение потерь", template="plotly_dark")
    display(fig)
    return fig


def update_loss_plot(fig, epoch, raw_loss, compressed_loss, fragment_loss):
    fig.data[0].x += (epoch,)
    fig.data[0].y += (raw_loss,)
    fig.data[1].x += (epoch,)
    fig.data[1].y += (compressed_loss,)
    fig.data[2].x += (epoch,)
    fig.data[2].y += (fragment_loss,)
    fig.show()


# Функция для расчета compression ratio
def calculate_compression_ratio(raw_embedding, compressed_embedding):
    """
    Вычисляет отношение нормы сжатого контекста к норме сырого контекста.
    """
    raw_norm = torch.norm(raw_embedding, p=2, dim=1)
    compressed_norm = torch.norm(compressed_embedding, p=2, dim=1)
    return compressed_norm / raw_norm


def predict_with_compressed_context(current_fragment, tokenizer, gpt2_model, device, max_new_tokens=50, num_beams=5):
    # Ensure the tokenizer is correctly initialized
    if isinstance(tokenizer, GPT2Tokenizer):
        # Set the pad_token to eos_token if not already set
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

        # Tokenize the current fragment
        inputs = tokenizer(current_fragment, return_tensors="pt", padding=True).to(device)
        input_ids = inputs.get("input_ids")
        attention_mask = inputs.get("attention_mask")  # Adding attention_mask

        # Check that input_ids is not empty
        if input_ids is not None and input_ids.shape[1] > 0:
            with torch.no_grad():
                # Generate predictions directly from current_fragment
                outputs = gpt2_model.generate(
                    input_ids=input_ids,
                    attention_mask=attention_mask,  # Use the attention mask
                    max_new_tokens=max_new_tokens,
                    num_beams=num_beams,
                    early_stopping=True,
                    pad_token_id=tokenizer.pad_token_id,  # Ensure pad_token_id is set correctly
                )
                predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
                return predicted_text
        else:
            print(f"Invalid inputs: input_ids length {input_ids.shape[1] if input_ids is not None else 'None'}")
    else:
        print("Invalid tokenizer object. Ensure the tokenizer is an instance of GPT2Tokenizer.")

    return "N/A"


# Использование предсказаний из предподготовленных данных
def evaluate_with_prepared_data(prepared_data, model, tokenizer, gpt2_model, device):
    """
    Выполняет оценку качества предсказаний с использованием подготовленных данных.
    """
    for entry in prepared_data:
        context = entry["context"]
        current_fragment = entry["current_fragment"]
        target = entry["target"]
        raw_prediction = entry["raw_prediction"]
        fragment_prediction = entry["fragment_prediction"]

        # Сжатие контекста и предсказание с ним
        compressed_context = model.compress(context)  # Предположим, у нас есть метод compress в модели
        compressed_prediction = predict_with_compressed_context(current_fragment, tokenizer, gpt2_model, device)

        # Сравнение предсказаний
        print(f"Контекст: {context}")
        print(f"Текущий фрагмент: {current_fragment}")
        print(f"Целевой текст: {target}")
        print(f"Предсказание по сырому контексту: {raw_prediction}")
        print(f"Предсказание по текущему фрагменту: {fragment_prediction}")
        print(f"Предсказание по сжатому контексту: {compressed_prediction}\n")


def load_intermediate_dataset(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

In [22]:
# Модель сжатия контекста

import torch
import torch.nn as nn
from torch.nn.functional import cosine_similarity


class ContextOptimizer(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=4, nhead=4):
        super(ContextOptimizer, self).__init__()
        self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model=input_size, nhead=nhead, dim_feedforward=hidden_size), num_layers=num_layers)
        self.compress_fc = nn.Linear(input_size, output_size)

    def forward(self, context):
        # Предполагается, что context имеет форму (seq_len, batch_size, input_size)
        encoded_context = self.encoder(context)
        compressed_context = self.compress_fc(encoded_context.mean(dim=0))  # Среднее по всем токенам
        return compressed_context


# Функция для сжатия контекста
def compress_context(context, optimizer, tokenizer, gpt2_model, device):
    inputs = tokenizer(context, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = gpt2_model(**inputs, output_hidden_states=True)
        raw_context_embedding = outputs.hidden_states[-1]  # Сохранение всех скрытых состояний

    # Подготовка предыстории для подачи в модуль сжатия
    context_embedding = raw_context_embedding.permute(1, 0, 2)  # Транспонируем для правильной формы
    compressed_context = optimizer(context_embedding)  # Получаем сжатый контекст

    return compressed_context, raw_context_embedding.mean(dim=1)  # Среднее значение как репрезентация предыстории

In [23]:
import time
import torch.optim as optim

from torch.nn.functional import cosine_similarity


def custom_loss(compressed_output, target_embedding, raw_context_embedding):
    """
    Функция потерь, которая учитывает схожесть с оригиналом и целевым вектором.
    """
    # Потеря восстановления: сравниваем сжатый контекст с исходным контекстом
    reconstruction_loss = 1 - cosine_similarity(compressed_output, raw_context_embedding, dim=1).mean()

    # Потеря предсказания: сравниваем предсказанный выход с целевым
    prediction_loss = nn.MSELoss()(compressed_output, target_embedding)

    # Итоговая потеря: комбинация восстановления и предсказания
    total_loss = prediction_loss + reconstruction_loss
    return total_loss


def combined_loss(compressed_output, target_embedding, full_context_output, regularization_factor=0.01):
    # Потеря предсказания
    prediction_loss = nn.MSELoss()(compressed_output, target_embedding)

    # Потеря восстановления для сохранения важной информации
    reconstruction_loss = 1 - cosine_similarity(compressed_output, full_context_output, dim=1).mean()

    # Регуляризация для предотвращения избыточности
    regularization_loss = regularization_factor * torch.norm(compressed_output, p=1)

    # Итоговая потеря
    total_loss = prediction_loss + reconstruction_loss + regularization_loss
    return total_loss


def train_context_optimizer(optimizer, intermediate_data, tokenizer, gpt2_model, epochs=4, device="cpu"):
    opt = optim.Adam(optimizer.parameters(), lr=0.001)
    raw_loss_values = []
    compressed_loss_values = []
    fragment_loss_values = []

    # Инициализация графика потерь
    loss_fig = create_loss_plot()
    # Для отслеживания потерь по каждому примеру на каждой эпохе
    example_loss_history = {i: [] for i in range(len(intermediate_data))}
    for epoch in range(epochs):
        total_compression_ratio = 0
        total_compressed_loss = 0
        total_divergence = 0  # Переменная для отслеживания расхождения между потерями
        count = 0

        epoch_losses = []  # Для хранения потерь текущей эпохи
        examples_calculated = []
        for idx, entry in enumerate(intermediate_data):
            context = entry["context"]
            current_fragment = entry["current_fragment"]
            target = entry["target"]

            # Используем предвычисленные значения потерь для сырого контекста и текущего фрагмента
            raw_loss = entry["raw_loss"]
            fragment_loss = entry["fragment_loss"]

            # Подготовка целевого предсказания
            target_input = tokenizer(target, return_tensors="pt").to(device)
            with torch.no_grad():
                target_output = gpt2_model(**target_input, output_hidden_states=True)
                target_embedding = target_output.hidden_states[-1].mean(dim=1)

            ## Сжатие контекста и вычисление потерь для сжатого контекста в комбинации с текущим фрагментом
            compressed_context, raw_context_embedding = compress_context(context, optimizer, tokenizer, gpt2_model, device)
            compressed_inputs = tokenizer(current_fragment, return_tensors="pt").to(device)
            compressed_context_repeated = compressed_context.unsqueeze(1).repeat(1, compressed_inputs["input_ids"].size(1), 1)

            # Преобразование размерности с помощью линейного слоя, чтобы совпадало с GPT-2
            compressed_context_resized = torch.nn.Linear(compressed_context.size(-1), gpt2_model.config.hidden_size)(compressed_context_repeated)

            # Сложение преобразованных тензоров
            compressed_inputs_embeds = compressed_context_resized + gpt2_model.transformer.wte(compressed_inputs["input_ids"])
            compressed_outputs = gpt2_model(inputs_embeds=compressed_inputs_embeds, output_hidden_states=True).hidden_states[-1].mean(dim=1)

            # compressed_loss = criterion(compressed_outputs, target_embedding)
            # Используем combined_loss для оптимизации
            loss = combined_loss(compressed_outputs, target_embedding, raw_context_embedding)

            # Обновление оптимизатора только по скорректированным потерям
            opt.zero_grad()
            loss.backward()
            opt.step()

            total_compressed_loss += loss.item()
            count += 1

            # Сохраняем текущую потерю для анализа изменения потерь
            epoch_losses.append(loss.item())
            example_loss_history[idx].append(loss.item())
            # Сохранение примеров для диагностики
            with torch.no_grad():
                compressed_pred = predict_with_compressed_context(current_fragment, tokenizer, gpt2_model, device)
                raw_pred = entry["raw_prediction"]
                fragment_pred = entry["fragment_prediction"]

                # Проверяем и обрабатываем compressed_context корректно перед декодированием
                if compressed_context is not None and isinstance(compressed_context, torch.Tensor):
                    # Пытаемся преобразовать в список токенов, исключая None
                    compressed_tokens = [token for token in compressed_context.squeeze().tolist() if isinstance(token, (int, float)) and 0 <= token < tokenizer.vocab_size]

                    if compressed_tokens:
                        # Декодируем только корректные значения токенов
                        compressed_context_text = tokenizer.decode(compressed_tokens)
                    else:
                        compressed_context_text = "Empty or Invalid tokens found in compressed context"
                else:
                    compressed_context_text = "N/A"

                # Сохраняем примеры в отдельный список для всех примеров
                examples_calculated.append(
                    {
                        "context": context,
                        "compressed_context": compressed_context_text,
                        "current_fragment": current_fragment,
                        "compressed_prediction": compressed_pred,
                        "raw_prediction": raw_pred,
                        "fragment_prediction": fragment_pred,
                        "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                    }
                )

        # Анализ потерь между эпохами для вывода примеров с максимальным снижением потерь
        if epoch > 0:
            loss_reduction = [
                (i, example_loss_history[i][-2] - example_loss_history[i][-1], examples_calculated[i]["compressed_context"], examples_calculated[i]["compressed_prediction"])
                for i in range(len(epoch_losses))
                if len(example_loss_history[i]) > 1
            ]
            loss_reduction.sort(key=lambda x: x[1], reverse=True)
            top_examples = loss_reduction[:3]  # Три примера с максимальным снижением потерь
        else:
            top_examples = [
                (i, 0, examples_calculated[i]["compressed_context"], examples_calculated[i]["compressed_prediction"]) for i in range(min(3, len(examples_calculated)))
            ]  # Первые три примера для первой эпохи

        avg_compressed_loss = total_compressed_loss / count if count > 0 else 0
        avg_compression_ratio = total_compression_ratio / count if count > 0 else 0
        avg_divergence = total_divergence / count if count > 0 else 0
        raw_loss_values.append(raw_loss)
        compressed_loss_values.append(avg_compressed_loss)
        fragment_loss_values.append(fragment_loss)

        print(f"Epoch {epoch+1}, Avg Compressed Context Loss: {avg_compressed_loss:.4f}, Avg Compression Ratio: {avg_compression_ratio:.4f}, Avg Divergence: {avg_divergence:.4f}")

        # Обновление графика после каждой эпохи
        update_loss_plot(loss_fig, epoch + 1, raw_loss, avg_compressed_loss, fragment_loss)

        # Вывод примеров с максимальным снижением потерь
        print(f"\n=== Примеры с максимальным снижением потерь после эпохи {epoch+1} ===")
        for idx, reduction, compressed_context, compressed_prediction in top_examples:
            example = intermediate_data[idx]
            print(f"\nПример {idx+1} (Снижение потерь: {reduction:.4f}):")
            print(f"Предыстория: {example['context']}")
            print(f"Сжатый контекст: {compressed_context}")  # Используем сохраненный сжатый контекст
            print(f"Текущий фрагмент: {example['current_fragment']}")
            print(f"Предсказание по сжатому контексту: {compressed_prediction}")  # Используем сохраненное предсказание по сжатому контексту
            print(f"Предсказание по сырому контексту: {example['raw_prediction']}")
            print(f"Предсказание по текущему фрагменту: {example['fragment_prediction']}")
            print(f"Целевой текст: {example['target']}")

    return raw_loss_values, compressed_loss_values, fragment_loss_values

In [24]:
import torch.optim as optim
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Настройки устройства
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загрузка модели GPT-2 и токенизатора
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

if not isinstance(tokenizer, GPT2Tokenizer):
    raise ValueError("The tokenizer must be an instance of GPT2Tokenizer.")

# Проверка и установка pad_token для корректной работы с padding
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Устанавливаем pad_token как eos_token
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    gpt2_model.resize_token_embeddings(len(tokenizer))  # Изменяем размер словаря модели после добавления токенов

# Загрузка датасета и подготовка промежуточных данных
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
intermediate_dataset_path = "intermediate_dataset.json"

# Генерация промежуточного датасета, если он еще не создан
if not os.path.exists(intermediate_dataset_path):
    generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device=device, intermediate_dataset_path=intermediate_dataset_path)
else:
    # Загрузка промежуточного датасета
    intermediate_data = load_intermediate_dataset(intermediate_dataset_path)

# Инициализация модели сжатия контекста
input_size = 768  # Размер входных данных модели GPT-2
hidden_size = 256  # Скрытый размер для оптимизатора контекста
output_size = 30
context_optimizer = ContextOptimizer(input_size, hidden_size, output_size).to(device)

# Загрузка сохраненной модели, если она существует
load_model(context_optimizer, context_optimizer, "context_optimizer3.pth")

# Запуск обучения
raw_loss_values, compressed_loss_values, fragment_loss_values = train_context_optimizer(context_optimizer, intermediate_data, tokenizer, gpt2_model, epochs=10, device=device)

# Сохранение модели после обучения
save_model(context_optimizer, context_optimizer, "context_optimizer3.pth")

Using device: cpu
Model file not found at context_optimizer2.pth


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Raw Context Loss',
              'type': 'scatter',
              'uid': '408d0e58-61a6-4aab-94f6-55bac9469eec',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'name': 'Compressed Context Loss',
              'type': 'scatter',
              'uid': '51abc40e-73e0-4555-900f-e694e1d56ed0',
              'x': [],
              'y': []},
             {'mode': 'lines+markers',
              'name': 'Fragment Only Loss',
              'type': 'scatter',
              'uid': 'e634a77d-ce54-4ceb-b8f3-39c7d9ba14be',
              'x': [],
              'y': []}],
    'layout': {'template': '...',
               'title': {'text': 'Графики потерь во время обучения'},
               'xaxis': {'title': {'text': 'Эпоха'}},
               'yaxis': {'title': {'text': 'Среднее значение потерь'}}}
})

Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenizer padding token: [PAD]
Tokenize


=== Примеры с максимальным снижением потерь после эпохи 1 ===

Пример 1 (Снижение потерь: 0.0000):
Предыстория: = = Development = =
Сжатый контекст: "!!!!!!!!"!!!!!"!
Текущий фрагмент: Concept work for Valkyria Chronicles III began after development finished on Valkyria Chronicles II in early 2010 , with full development beginning shortly after this . The director of Valkyria Chronicles II , Takeshi Ozawa , returned to that role for Valkyria Chronicles III . Development work took approximately one year . After the release of Valkyria Chronicles II , the staff took a look at both the popular response for the game and what they wanted to do next for the series . Like its predece
Предсказание по сжатому контексту: Concept work for Valkyria Chronicles III began after development finished on Valkyria Chronicles II in early 2010, with full development beginning shortly after this. The director of Valkyria Chronicles II, Takeshi Ozawa, returned to that role for Valkyria Chronicles III. Devel


=== Примеры с максимальным снижением потерь после эпохи 2 ===

Пример 54 (Снижение потерь: 16.7909):
Предыстория: = = = January – February = = =
Сжатый контекст: !#$!#%"&%"!!&""#"(
Текущий фрагмент: With the losing continuing , more rumors began to surface . Unlike before , the rumors were about player moves rather than coaching changes . The majority of rumors were that the Blue Jackets would trade Rick Nash . While Howson stated that he had never brought up trading Nash in discussions , other teams had inquired about his availability . Nash stated that if Columbus felt it would make the franchise better than he would be willing to waive his no @-@ trade clause . Howson publicly stated tha
Предсказание по сжатому контексту: With the losing continuing, more rumors began to surface. Unlike before, the rumors were about player moves rather than coaching changes. The majority of rumors were that the Blue Jackets would trade Rick Nash. While Howson stated that he had never brought up trad

...


=== Примеры с максимальным снижением потерь после эпохи 6 ===

Пример 89 (Снижение потерь: 15.4657):
Предыстория: Divine behavior was believed to govern all of nature . Except for the few deities who disrupted the divine order , the gods ' actions maintained maat and created and sustained all living things . They did this work using a force the Egyptians called heka , a term usually translated as " magic " . Heka was a fundamental power that the creator god used to form the world and the gods themselves .
Сжатый контекст: $!!!"""$"#!"!!
Текущий фрагмент: The gods ' actions in the present are described and praised in hymns and funerary texts . In contrast , mythology mainly concerns the gods ' actions during a vaguely imagined past in which the gods were present on earth and interacted directly with humans . The events of this past time set the pattern for the events of the present . Periodic occurrences were tied to events in the mythic past ; the succession of each new pharaoh , for 


=== Примеры с максимальным снижением потерь после эпохи 7 ===

Пример 86 (Снижение потерь: 17.3489):
Предыстория: Predynastic Egypt originally consisted of small , independent villages . Because many deities in later times were strongly tied to particular towns and regions , many scholars have suggested that the pantheon formed as disparate communities coalesced into larger states , spreading and intermingling the worship of the old local deities . But others have argued that the most important predynastic gods were , like other elements of Egyptian culture , present all across the country despite the polit
Сжатый контекст: #!!#!$!"#""!#"%"
Текущий фрагмент: The final step in the formation of Egyptian religion was the unification of Egypt , in which rulers from Upper Egypt made themselves pharaohs of the entire country . These sacred kings and their subordinates assumed the exclusive right to interact with the gods , and kingship became the unifying focus of the religion .
Предсказани


=== Примеры с максимальным снижением потерь после эпохи 8 ===

Пример 184 (Снижение потерь: 16.5787):
Предыстория: = = Other notable caves = =
Сжатый контекст: !!$!"!"!!!"!!!
Текущий фрагмент: To the south @-@ east of the Great Cave is the second excavation , which faces east @-@ northeast . It includes a chapel at the north end . The front of this cave is completely destroyed ; only fragments of some semi @-@ columns remain . The interior has suffered water damage . The portico is 26 m ( 85 ft ) long and 11 m ( 36 ft ) deep . The chapel is supported by eight eight @-@ cornered columns and two demi @-@ columns and is irregular in shape . At the back of the portico are three chambers ;
Предсказание по сжатому контексту: To the south @-@ east of the Great Cave is the second excavation, which faces east @-@ northeast. It includes a chapel at the north end. The front of this cave is completely destroyed ; only fragments of some semi @-@ columns remain. The interior has suffered water dama


=== Примеры с максимальным снижением потерь после эпохи 9 ===

Пример 2 (Снижение потерь: 41.5035):
Предыстория: = = Reception = =
Сжатый контекст: !"##!"!""!!!
Текущий фрагмент: On its day of release in Japan , Valkyria Chronicles III topped both platform @-@ exclusive and multi @-@ platform sales charts . By early February , the game sold 102 @,@ 779 units , coming in second overall to The Last Story for the Wii . By the end of the year , the game had sold just over 152 @,@ 500 units .
Предсказание по сжатому контексту: On its day of release in Japan, Valkyria Chronicles III topped both platform @-@ exclusive and multi @-@ platform sales charts. By early February, the game sold 102 @,@ 779 units, coming in second overall to The Last Story for the Wii. By the end of the year, the game had sold just over 152 @,@ 500 units.
Предсказание по сырому контексту: 
 "gex. 0 0 = own = the = the. it =ries Chronicles 2: the charts and 1 and and to @--@ exclusive...
 comparison September, V game 


=== Примеры с максимальным снижением потерь после эпохи 10 ===

Пример 121 (Снижение потерь: 14.5311):
Предыстория: In January 1844 , North Carolina Representative James Iver McKay , the chairman of the Committee on Ways and Means , solicited the views of Director Patterson on the gold dollar . Patterson had more of Gobrecht 's pattern dollar struck to show to committee members , again advising against a coin that if issued would be only about a half inch ( 13 mm ) in diameter . He told Treasury Secretary John C. Spencer that the only gold coins of that size in commerce , the Spanish and Colombian half @-@ es
Сжатый контекст: ##!#"!!"!"!!"""
Текущий фрагмент: Even before 1848 , record amounts of gold were flowing to American mints to be struck into coin , but the California Gold Rush vastly increased these quantities . This renewed calls for a gold dollar , as well as for a higher denomination than the eagle ( $ 10 piece ) , then the largest gold coin . In January 1849 , McKay introdu

## Предподготовка данных

In [ ]:
import json
import torch
import time
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch.nn as nn


def generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device="cpu", intermediate_dataset_path="intermediate_dataset.json"):
    intermediate_data = []
    criterion = nn.MSELoss()

    # Подготавливаем данные
    print("Начало подготовки данных...")
    prepared_data = prepare_data(dataset, max_length=500)

    # Оценка времени выполнения цикла для первых пяти итераций
    cycle_times = []
    for i, (context, current_fragment, target) in enumerate(prepared_data[:5]):
        if context is None or current_fragment is None:
            continue

        cycle_start_time = time.time()

        # Подготовка целевого предсказания
        target_input = tokenizer(target, return_tensors="pt").to(device)
        with torch.no_grad():
            target_output = gpt2_model(**target_input, output_hidden_states=True)
            target_embedding = target_output.hidden_states[-1].mean(dim=1)

        # Предсказание по сырому контексту (предыстория + текущий фрагмент)
        raw_inputs = tokenizer(context + " " + current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            raw_outputs = gpt2_model(**raw_inputs, output_hidden_states=True)
            raw_combined_embedding = raw_outputs.hidden_states[-1].mean(dim=1)
            raw_loss = criterion(raw_combined_embedding, target_embedding).item()
            raw_prediction = tokenizer.decode(raw_outputs.logits.argmax(-1).squeeze().tolist())

        # Предсказание только по текущему фрагменту
        fragment_input = tokenizer(current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            fragment_output = gpt2_model(**fragment_input, output_hidden_states=True)
            fragment_embedding = fragment_output.hidden_states[-1].mean(dim=1)
            fragment_loss = criterion(fragment_embedding, target_embedding).item()
            fragment_prediction = tokenizer.decode(fragment_output.logits.argmax(-1).squeeze().tolist())

        cycle_end_time = time.time()
        cycle_times.append(cycle_end_time - cycle_start_time)

        # Фильтрация по критерию fragment_loss > raw_loss
        if fragment_loss > raw_loss and fragment_loss < 1:
            # Сохранение всех значений и предсказаний
            intermediate_data.append(
                {
                    "context": context,
                    "current_fragment": current_fragment,
                    "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                    "raw_loss": raw_loss,
                    "fragment_loss": fragment_loss,
                    "raw_prediction": raw_prediction,
                    "fragment_prediction": fragment_prediction,
                }
            )

    # Оценка общего времени выполнения цикла на основе первых пяти измерений
    avg_cycle_time = sum(cycle_times) / len(cycle_times) if cycle_times else 0
    estimated_total_time = avg_cycle_time * len(prepared_data)
    print(f"Оценка времени выполнения цикла: {estimated_total_time:.2f} секунд для {len(prepared_data)} записей.")

    # Продолжение обработки остальных данных после оценки времени
    for i, (context, current_fragment, target) in enumerate(prepared_data[5:]):
        if context is None or current_fragment is None:
            continue

        # Подготовка целевого предсказания
        target_input = tokenizer(target, return_tensors="pt").to(device)
        with torch.no_grad():
            target_output = gpt2_model(**target_input, output_hidden_states=True)
            target_embedding = target_output.hidden_states[-1].mean(dim=1)

        # Предсказание по сырому контексту (предыстория + текущий фрагмент)
        raw_inputs = tokenizer(context + " " + current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            raw_outputs = gpt2_model(**raw_inputs, output_hidden_states=True)
            raw_combined_embedding = raw_outputs.hidden_states[-1].mean(dim=1)
            raw_loss = criterion(raw_combined_embedding, target_embedding).item()
            raw_prediction = tokenizer.decode(raw_outputs.logits.argmax(-1).squeeze().tolist())

        # Предсказание только по текущему фрагменту
        fragment_input = tokenizer(current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            fragment_output = gpt2_model(**fragment_input, output_hidden_states=True)
            fragment_embedding = fragment_output.hidden_states[-1].mean(dim=1)
            fragment_loss = criterion(fragment_embedding, target_embedding).item()
            fragment_prediction = tokenizer.decode(fragment_output.logits.argmax(-1).squeeze().tolist())

        # Фильтрация по критерию fragment_loss > raw_loss and fragment_loss <= 1
        if fragment_loss > raw_loss and fragment_loss < 1:
            # Сохранение всех значений и предсказаний
            intermediate_data.append(
                {
                    "context": context,
                    "current_fragment": current_fragment,
                    "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                    "raw_loss": raw_loss,
                    "fragment_loss": fragment_loss,
                    "raw_prediction": raw_prediction,
                    "fragment_prediction": fragment_prediction,
                }
            )

    # Сохранение промежуточного датасета в JSON
    with open(intermediate_dataset_path, "w") as f:
        json.dump(intermediate_data, f, indent=4)

    print(f"Промежуточный датасет создан с {len(intermediate_data)} записями, включая значения потерь и предсказания.")


# Настройки
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загружаем датасет wikitext-2-raw-v1
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Инициализация токенизатора и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)
# Set the pad_token to eos_token if not already set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token
    tokenizer.add_special_tokens({"pad_token": "[PAD]"})
    gpt2_model.resize_token_embeddings(len(tokenizer))  # Update model embeddings after modifying the tokenizer

# Задаем путь для сохранения промежуточного датасета
intermediate_dataset_path = "intermediate_dataset.json"

# Генерация промежуточного датасета
generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device, intermediate_dataset_path)

Using device: cpu
Начало подготовки данных...
Оценка времени выполнения цикла: 435.63 секунд для 999 записей.
Промежуточный датасет создан с 197 записями, включая значения потерь и предсказания.


In [ ]:
# Устарело
import json
import torch
import time
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from datasets import load_dataset
import torch.nn as nn


def generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device="cpu", intermediate_dataset_path="intermediate_dataset.json"):
    intermediate_data = []
    criterion = nn.MSELoss()

    # Подготовка данных
    print("Начало подготовки данных...")
    start_time = time.time()
    prepared_data = prepare_data(dataset, max_length=500)
    print("Данные подгружены.")
    # Оценка времени подготовки данных
    preparation_times = []
    for i, (context, current_fragment, target) in enumerate(prepared_data[:5]):
        if context is None or current_fragment is None:
            continue
        preparation_times.append(time.time() - start_time)
        start_time = time.time()

    avg_preparation_time = sum(preparation_times) / len(preparation_times)
    estimated_total_time = avg_preparation_time * len(prepared_data)
    print(f"Оценка времени подготовки данных: {estimated_total_time:.2f} секунд для {len(prepared_data)} записей.")

    for i, (context, current_fragment, target) in enumerate(prepared_data):
        if context is None or current_fragment is None:
            continue

        # Подготовка целевого предсказания
        target_input = tokenizer(target, return_tensors="pt").to(device)
        with torch.no_grad():
            target_output = gpt2_model(**target_input, output_hidden_states=True)
            target_embedding = target_output.hidden_states[-1].mean(dim=1)

        # Предсказание по сырому контексту (предыстория + текущий фрагмент)
        raw_inputs = tokenizer(context + " " + current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            raw_outputs = gpt2_model(**raw_inputs, output_hidden_states=True)
            raw_combined_embedding = raw_outputs.hidden_states[-1].mean(dim=1)
            raw_loss = criterion(raw_combined_embedding, target_embedding).item()

        # Предсказание только по текущему фрагменту
        fragment_input = tokenizer(current_fragment, return_tensors="pt").to(device)
        with torch.no_grad():
            fragment_output = gpt2_model(**fragment_input, output_hidden_states=True)
            fragment_embedding = fragment_output.hidden_states[-1].mean(dim=1)
            fragment_loss = criterion(fragment_embedding, target_embedding).item()

        # Сохранение всех значений и предсказаний
        intermediate_data.append(
            {
                "context": context,
                "current_fragment": current_fragment,
                "target": tokenizer.decode(target_input["input_ids"].squeeze().tolist()),
                "raw_loss": raw_loss,
                "fragment_loss": fragment_loss,
            }
        )

    # Сохранение промежуточного датасета в JSON
    with open(intermediate_dataset_path, "w") as f:
        json.dump(intermediate_data, f, indent=4)

    print(f"Промежуточный датасет создан с {len(intermediate_data)} записями, включая значения потерь.")


# Настройки
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Загружаем датасет wikitext-2-raw-v1
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")

# Инициализация токенизатора и модели GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2").to(device)  # Используем 'cpu', можно заменить на 'cuda' при использовании GPU

# Задаем путь для сохранения промежуточного датасета
intermediate_dataset_path = "intermediate_dataset.json"

# Генерация промежуточного датасета
generate_intermediate_dataset(dataset, tokenizer, gpt2_model, device, intermediate_dataset_path)

Using device: cpu


c:\git\MUIV\concl\.venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Начало подготовки данных...
Данные подгружены.
Оценка времени подготовки данных: 276.26 секунд для 999 записей.
Промежуточный датасет создан с 999 записями, включая значения потерь.
